## Load Base Stable Diffusion Model

In [ ]:
from diffusers import StableDiffusionPipeline
from pruna.algorithms.smasher_config import AutoSmasherConfig
from pruna.smash import smash
import torch
import requests
from PIL import Image
from io import BytesIO

from diffusers import StableDiffusionInpaintPipeline

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float16,
)
image = Image.open(
            BytesIO(
                requests.get(
                    "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
                ).content
            )
        )
mask_image = Image.open(
            BytesIO(
                requests.get(
                    "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"
                ).content
            )
        )
prompt = "cat"
#image and mask_image should be PIL images.
#The mask structure is white for inpainting and black for keeping as is
pipe.to('cuda')
# image = pipe(prompt=prompt, image=image, mask_image=mask_image).images[0]

## Smash it!

In [ ]:
from pruna.algorithms.smasher_config import SmasherConfig

smasher_config = SmasherConfig()
smasher_config['compiler'] = 'diffusers'
smasher_config['max_batch_size'] = 1
smasher_config['image_height'] = 512
smasher_config['image_width'] = 512
smasher_config['version'] = '1.5'
smasher_config['static_shape'] = False
smasher_config['unet_dim'] = 9
smasher_config['scheduler'] = "PNDM"

In [ ]:
smashed_model = smash(
        model=pipe,
        data_module="inpaint",
        api_key='your-api-key',
        model_config=None,
        smasher_config=smasher_config,
        device='cuda',
    )

## Base Model Generation

In [ ]:
%%time
pipe(prompt=prompt, image=image, mask_image=mask_image).images[0]

## Smashed Model Generation

In [ ]:
%%time
smashed_model(prompt, input_image=image, mask_image=mask_image, image_height=512, image_width=512, strength=0.75, num_inference_steps=50)[0]